In [1]:
import urllib
import pickle
import io
import os
import csv
import numpy as np

nof_pdf_files = 3


In [2]:
print ('reading arxiv categories ...')

catlist = []
with io.open('ArxivSubjectCategory.csv', newline='\n') as csvfile:
    catreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    catlist = [row[0] for row in catreader]

print (len(catlist), 'categories read.')
print ('sample:', np.random.choice(catlist), ',', np.random.choice(catlist), ',', np.random.choice(catlist))

reading arxiv categories ...
153 categories read.
sample: cs.CY , math.GM , nucl-ex


In [3]:
print ('generating queries ...')
q = 'http://export.arxiv.org/api/query?search_query=cat:{cat}&all:{yr}&start=0&max_results={n}'
yrlist = list(range(2001, 2019)) # from 2001 to 2018
qrylist = []

for yr in yrlist:
    for cat in catlist:
        qry = q.format(cat=cat, yr=yr, n=nof_pdf_files)
        qrylist.append(qry)
        
print (len(yrlist), 'years, ', len(catlist), 'categories, ', 'total', len(qrylist), 'queries.')
print ('sample:', np.random.choice(qrylist))

generating queries ...
18 years,  153 categories,  total 2754 queries.
sample: http://export.arxiv.org/api/query?search_query=cat:physics.class-ph&all:2007&start=0&max_results=3


In [10]:
def make_query(query):
    return urllib.request.urlopen(query).read()

def query_and_store(query, pickle_fpath='data.pickle', force=False):
    # read data(dict) from pickle, otherwise create a new dict
    try:
        f = open(pickle_fpath, 'rb')
        data = pickle.load(f)
        f.close()
    except IOError:
        data = {}

    # if query not found in local data or forced, do query and save back to the pickle file
    if type(query) is list:
        print ('is a list')
        tobequery = set(query) - set(data) # compute query which is not in data
        if len(tobequery) != 0 or force: # not found in data{}, need some query
            for qry in tobequery:
                data[qry] = make_query(qry)
                print ('.', end='')
            pickle.dump(data, open(pickle_fpath, 'wb'))
        
        return {qry : data[qry] for qry in query}
        
    elif type(query) is str: # not a list, should be a string
        print ('is a str')
        if query not in data:
            data[query] = make_query(query)
            pickle.dump(data, open(pickle_fpath, 'wb'))
        
        return data[query]
    
    else:
        print ('is else')
        return ''

In [11]:
data = query_and_store(qrylist)
print (len(data), len(qrylist))
for d in data:
    print (data[d])
    break

is a list
........................................

KeyboardInterrupt: 

In [50]:
a = {'a':'asdf', 'z':'zxcv', 's':'sdfg'}
b = {'a':'asdf', 'z':'zxcv', 'q':'qwer'}

In [51]:
print (set(b)-set(a))

{'q'}
